In [1]:
!nvidia-smi

Tue Oct 17 20:15:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

## 모델 로딩

In [3]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

# 데이터 전처리

In [4]:
# import json

# with open ("./law_page1to50.json", "r") as f:
#     data_1 = json.load(f)
# print(data_1)

## LsTrmSearch 풀기

In [5]:
# list = []
# for i in data_1:
#   a_list = i['LsTrmSearch']
#   list.append(a_list)
# print(list)

### 저장하고 불러오기

In [6]:
# with open("./law_re.json", 'w', encoding='utf-8') as json_file:
#     json.dump(list, json_file)

# with open("./law_re.json", "r") as json_new_file:
#   json_data_re = json.load(json_new_file)

In [7]:
# print(json_data_re)

## lstrm 풀기

In [8]:
# list = []
# for i in json_data_re:
#   a_list = i['lstrm']
#   list.append(a_list)
# print(list)

### 이중 리스트 풀기

In [9]:
# list_1=[]
# for i in list:
#   for j in i:
#     list_1.append(j)
# print(list_1)

### 저장하고 불러오기

In [10]:
# with open("./law_rere.json", 'w', encoding='utf-8') as json_file:
#     json.dump(list_1, json_file, ensure_ascii=False)

# with open("./law_rere.json", "r") as json_new_file:
#   json_data_rere = json.load(json_new_file)

In [11]:
# json_data_rere

# 데이터 로딩

In [12]:
from datasets import load_dataset

data = load_dataset("json",
                    data_files={"train":"law_rere.json"})
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['법령용어상세링크', '법령용어명', '사전구분코드', '법령용어상세검색', '법령종류코드', '법령용어ID', '@id'],
        num_rows: 1000
    })
})

In [13]:
[row for row in data['train']]

[{'법령용어상세링크': '/DRF/lawService.do?OC=duja1017&target=lstrm&trmSeqs=3945293&mobile=&type=XML',
  '법령용어명': '(Instrument Meterological Condition; IMC)',
  '사전구분코드': '011402',
  '법령용어상세검색': ' 시계비행 기상상태외의 기상상태를 말한다. ',
  '법령종류코드': '010102',
  '법령용어ID': '3945293',
  '@id': '1'},
 {'법령용어상세링크': '/DRF/lawService.do?OC=duja1017&target=lstrm&trmSeqs=4043616&mobile=&type=XML',
  '법령용어명': '(Terminal Operating Company)라 한다)',
  '사전구분코드': '011402',
  '법령용어상세검색': ' 항만시설운영자와 임대계약을 체결한 자(이하 "임대계약자"라 한다)로서 선석ㆍ보관시설ㆍ하역시설 등 부두시설 일체를 전용(專用)사용하는 민간업체를 말한다. 다만, 컨테이너부두와 개발부두의 임대계약자, 항만시설운영자가 인정하는 임대계약자는 제외한다. ',
  '법령종류코드': '010101',
  '법령용어ID': '4043616',
  '@id': '2'},
 {'법령용어상세링크': '/DRF/lawService.do?OC=duja1017&target=lstrm&trmSeqs=4472280&mobile=&type=XML',
  '법령용어명': '(이하',
  '사전구분코드': '011402',
  '법령용어상세검색': ' 인터넷을 통한 화학물질 불법유통 및 폭발물 제조방법 등 유해정보를 모니터링, 신고하는 활동에 자발적으로 참여하도록 위촉된 자를 말한다. ',
  '법령종류코드': '010102',
  '법령용어ID': '4472280',
  '@id': '3'},
 {'법령용어상세링크': '/DRF/lawService.do?OC=duja1017&target=lstr

In [14]:
data = data.map(
    lambda x: {
        'text':
        f"""User: {x['법령용어명']}

Assistant: {x['법령용어상세검색']}<|endoftext|>"""
        }
) #instruction, input, output

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
[row['text'] for row in data['train']]

['User: (Instrument Meterological Condition; IMC)\n\nAssistant:  시계비행 기상상태외의 기상상태를 말한다. <|endoftext|>',
 'User: (Terminal Operating Company)라 한다)\n\nAssistant:  항만시설운영자와 임대계약을 체결한 자(이하 "임대계약자"라 한다)로서 선석ㆍ보관시설ㆍ하역시설 등 부두시설 일체를 전용(專用)사용하는 민간업체를 말한다. 다만, 컨테이너부두와 개발부두의 임대계약자, 항만시설운영자가 인정하는 임대계약자는 제외한다. <|endoftext|>',
 'User: (이하\n\nAssistant:  인터넷을 통한 화학물질 불법유통 및 폭발물 제조방법 등 유해정보를 모니터링, 신고하는 활동에 자발적으로 참여하도록 위촉된 자를 말한다. <|endoftext|>',
 'User: (지리적)위치(Position(geographical))\n\nAssistant:  지구 표면상의 특정 지점에 대한 위치를 규정하는 수학적 기준타원체를 기준으로 표시한 일련의 좌표집합(위도 및 경도)을 말한다. <|endoftext|>',
 'User: 0 정지방식 비상정지장치\n\nAssistant:  액츄에이터(actuator)의 전원차단방식에 의하여 정지되는 비상정지장치를 말한다.12. 사출성형기의 "주요 구조부"는 다음 각 목과 같다.가. 형체기구 및 사출기구나. 프레임(frame)다. 방호장치라. 유ㆍ공압 계통마. 제어반 <|endoftext|>',
 'User: 0 정지방식(stop category 0) 비상정지장치\n\nAssistant:  액츄에이터(actuator)의 전원차단방식에 의해 정지되는 비상정지장치를 말한다. <|endoftext|>',
 'User: 080착신과금사업자\n\nAssistant:  착신과금서비스를 제공하는 기간통신사업자를 말한다. <|endoftext|>',
 'User: 080착신과금서비스\n\nAssistant: 「전기통신사업법」 제2조제11

## 텍스트 데이터 tokenize

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
data["train"][0]["text"]

'User: (Instrument Meterological Condition; IMC)\n\nAssistant:  시계비행 기상상태외의 기상상태를 말한다. <|endoftext|>'

In [18]:
tokenized_text = tokenizer(data["train"][0]["text"])
tokenized_text

{'input_ids': [56, 426, 85, 29, 1585, 13801, 3864, 85, 9990, 6522, 1663, 5145, 2423, 23315, 21760, 1442, 21836, 26743, 30, 1709, 7442, 12, 202, 202, 36, 12732, 15097, 15741, 29, 224, 6810, 10966, 5547, 5464, 872, 285, 5547, 5464, 301, 502, 660, 17, 224, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
token_text = [tokenizer.decode(id) for id in tokenized_text["input_ids"]]
", ".join(token_text)

'U, se, r, :,  (, In, st, r, um, ent,  M, et, er, olog, ical,  C, ond, ition, ;,  I, MC, ), \n, \n, A, ss, ist, ant, :,  ,  시계, 비행,  기상, 상태, 외, 의,  기상, 상태, 를,  말, 한다, .,  , <|endoftext|>'

## PEFT 학습 준비

In [20]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [21]:
# 학습되는 파라미터 수 출력
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"""trainable params: {trainable_params}
all params: {all_param}
trainable%: {100 * trainable_params / all_param}""")

trainable params: 6553600
all params: 6608701440
trainable%: 0.09916622894073424


## 학습하기

In [22]:
import transformers

tokenizer.pad_token = tokenizer.eos_token 
# 문장의 패딩값을 eos(end of sequence) 토큰으로 변경 
# => 패딩이 들어오면 문장이 끝나는 형태로 토크나이징

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        max_steps=500, # 100 step 학습시, V100 기준 약 2분 소요
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        learning_rate=1e-4,
        fp16=True,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        logging_steps=10,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False # silence the warnings. Please re-enable for inference!
trainer.train()

# logging_steps = 10
# max_steps

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,4.214700
20,3.316400
30,2.671500
40,2.363400
50,2.186400
60,1.730800
70,1.778600
80,1.866800
90,1.593400
100,1.482800


TrainOutput(global_step=500, training_loss=1.494330457687378, metrics={'train_runtime': 1567.4555, 'train_samples_per_second': 0.638, 'train_steps_per_second': 0.319, 'total_flos': 4421740654878720.0, 'train_loss': 1.494330457687378, 'epoch': 1.0})

In [23]:
model.eval()
model.config.use_cache = True

In [24]:
from transformers.generation import GenerationConfig

gen_cfg = GenerationConfig.from_model_config(model.config)
gen_cfg.pad_token_id = tokenizer.eos_token_id

def gen(x):
    gen_cfg.temperature = 0.1
    gen_cfg.max_new_tokens = 256

    gened = model.generate(
        **tokenizer(
            f"User: {x}\n\nAssistant:",
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        do_sample=True,
        generation_config=gen_cfg,
    )

    return tokenizer.decode(gened[0])

In [25]:
# eos_token_id=2,

In [26]:
gen('8자형 링이 무엇인가요?')

torch.Size([1, 18])
no pkv
torch.Size([1, 18])
torch.Size([1, 19])
torch.Size([1, 40, 18, 128])
torch.Size([1, 1])
torch.Size([1, 20])
torch.Size([1, 40, 19, 128])
torch.Size([1, 1])
torch.Size([1, 21])
torch.Size([1, 40, 20, 128])
torch.Size([1, 1])
torch.Size([1, 22])
torch.Size([1, 40, 21, 128])
torch.Size([1, 1])
torch.Size([1, 23])
torch.Size([1, 40, 22, 128])
torch.Size([1, 1])
torch.Size([1, 24])
torch.Size([1, 40, 23, 128])
torch.Size([1, 1])
torch.Size([1, 25])
torch.Size([1, 40, 24, 128])
torch.Size([1, 1])
torch.Size([1, 26])
torch.Size([1, 40, 25, 128])
torch.Size([1, 1])
torch.Size([1, 27])
torch.Size([1, 40, 26, 128])
torch.Size([1, 1])
torch.Size([1, 28])
torch.Size([1, 40, 27, 128])
torch.Size([1, 1])
torch.Size([1, 29])
torch.Size([1, 40, 28, 128])
torch.Size([1, 1])
torch.Size([1, 30])
torch.Size([1, 40, 29, 128])
torch.Size([1, 1])
torch.Size([1, 31])
torch.Size([1, 40, 30, 128])
torch.Size([1, 1])
torch.Size([1, 32])
torch.Size([1, 40, 31, 128])
torch.Size([1, 1])
t

'User: 8자형 링이 무엇인가요?\n\nAssistant:  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형 링은 원형의 링을 8개의 조각으로 분할하여 제작한 것을 말한다.  8자형'

endoftext 이 안나왔는데 나오려면 1000개 이상의 데이터셋을 학습시켜야 함

## 모델 저장 & 업로드

In [27]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [28]:
model.push_to_hub('julietz/koalpaca-polyglot-12.8b-law')

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/julietz/koalpaca-polyglot-12.8b-law/commit/3c97041ef02e8ca79387075274784474dfa786c2', commit_message='Upload model', commit_description='', oid='3c97041ef02e8ca79387075274784474dfa786c2', pr_url=None, pr_revision=None, pr_num=None)

Reference: [KoAlpaca QLoRA Training Original Notebook by Beomi](https://colab.research.google.com/gist/Beomi/a3032e4eaa33b86fdf8de1f47f15a647/2023_05_26_bnb_4bit_koalpaca_v1_1a_on_polyglot_ko_12_8b.ipynb)